# Text Processing

Today, our goals are to explore one final set of methods that can help us processing text data--vectorized string methods in pandas--and then work through some realistic examples where we may need to combine multiple approaches:

* String methods
* Data transformation
* Regular expressions
* Vectorized string methods

Friendly Reminders:

* Project proposal due tonight by 11:59 p.m.
* DataCamp Module - Bringing it all together! (Python Data Science Toolbox), due March 12 by 11:59 p.m.
* Homework #4 due March 14 by 11:59 p.m.

In [1]:
import re
from numpy import nan as NA
import numpy as np
import pandas as pd
pd.set_option('max_colwidth',150)

## Vectorized String Methods in pandas

There is a large set of vectorized string methods available for Series (and Index) objects (but not DataFrames), summarized in the table below.

In [2]:
str_df = pd.read_html('https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html')[1].set_index('Method')
str_df.index = str_df.index.str.rstrip('()')
str_df.columns = ['']
str_df.sort_index().style.set_properties(**{'text-align': 'left'})

,
Method,
capitalize,Equivalent to str.capitalize
cat,Concatenate strings
center,Equivalent to str.center
contains,Return boolean array if each string contains pattern/regex
count,Count occurrences of pattern
endswith,Equivalent to str.endswith(pat) for each element
extract,"Call re.search on each element, returning DataFrame with one row for each element and one column for each regex capture group"
extractall,"Call re.findall on each element, returning DataFrame with one row for each match and one column for each regex capture group"
find,Equivalent to str.find


In [3]:
# Object introspection
pd.Series.str.split?

These methods are accessible via the .str attribute of the Series (or Index) object. Many of these methods are equivalent to our standard string methods, and work in the same way, except applied in a vectorized manner (i.e., you do not need to loop through each observation and apply the method to each value). In many cases, these methods are equivalent to applying a function to the Series (or Index) object via the .map method, but the vectorized methods are more flexible in that they do not raise Exceptions if you have missing data.

In [4]:
names = pd.Series(['anna','melody','jordan','william',NA])
names

0       anna
1     melody
2     jordan
3    william
4        NaN
dtype: object

In [5]:
# .map approach
names.map(lambda s: s.capitalize())
# missing value (NaN, a float object) cannot use capitalize

AttributeError: 'float' object has no attribute 'capitalize'

In [6]:
# .map approach with ternary expression
names.map(lambda s: s.capitalize() if pd.notnull(s) else s)
# map + lambda: kind of traditional

0       Anna
1     Melody
2     Jordan
3    William
4        NaN
dtype: object

In [7]:
# Vectorized method approach
names.str.capitalize()
# can handle NaN value more flexible

0       Anna
1     Melody
2     Jordan
3    William
4        NaN
dtype: object

There are multiple methods that can help with vectorized string indexing and slicing: 

* str.get(*index*) - equivalent to standard sequence indexing
* str.slice(*start*, *stop*, *step*) - equivalent to standard sequence slicing
* Standard .str[*index*] indexing and .str[*start*:*stop*:*step*] slicing

In [8]:
names.str.get(0)

0      a
1      m
2      j
3      w
4    NaN
dtype: object

In [9]:
names.str[0]

0      a
1      m
2      j
3      w
4    NaN
dtype: object

In [10]:
names.str.get(-1)

0      a
1      y
2      n
3      m
4    NaN
dtype: object

In [11]:
names.str.slice(0,5,2)

0     an
1    mld
2    jra
3    wli
4    NaN
dtype: object

In [12]:
names.str[:3]

0    ann
1    mel
2    jor
3    wil
4    NaN
dtype: object

Several vectorized string methods are compatible with regular expressions:

* str.contains(*pat*, *case*=True, *flags*=0) - Returns a Boolean if a match is found (similar to our check_re function that uses re.search)
* str.extract(*pat*, *flags*=0, *expand*=None) - Returns the first found match
* str.extractall(*pat*, *flags*=0) - Returns all matches in a hierarchically indexed DataFrame
* str.findall(*pat*, *flags*=0) - Returns all matches as a single list
* str.match(*pat*, *case*=True, *flags*=0) - Similar to str.contains, but based on re.match
* str.replace(*pat*, *repl*, _n_=-1, *case*=None, *flags*=0) - Similar to re.sub/re.subn
* str.split(*pat*, _n_=-1, *case*=None, *expand*=False) - Similar to re.split

As you can see, many of these methods have similar arguments, including:

* *pat* - The literal or regular expression that you want to match (string); you can also input compiled regexes
* *case* - A flag determining whether the match should be case sensitive or not (can also be controlled via re.IGNORECASE)
* *flags* - Argument that access re control flags (e.g., re.IGNORECASE)
* *repl* - Replacement string or function to apply to matched text
* *n* - Limit on number of operations (e.g., replacements, splits); defaults to -1 (no limit)
* *expand* - A flag determining whether expand returning object into a DataFrame (True) or a Series/Index (False) object

In [13]:
# Create date range
dser = pd.date_range(start='1/1/2018', end='12/31/2018', freq='2W').astype(str)
dser

Index(['2018-01-07', '2018-01-21', '2018-02-04', '2018-02-18', '2018-03-04',
       '2018-03-18', '2018-04-01', '2018-04-15', '2018-04-29', '2018-05-13',
       '2018-05-27', '2018-06-10', '2018-06-24', '2018-07-08', '2018-07-22',
       '2018-08-05', '2018-08-19', '2018-09-02', '2018-09-16', '2018-09-30',
       '2018-10-14', '2018-10-28', '2018-11-11', '2018-11-25', '2018-12-09',
       '2018-12-23'],
      dtype='object')

In [14]:
# str.contains
dser.str.contains('\d{4}')

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True])

In [15]:
# str.extract - Single group
dser.str.extract('(\d\d)', expand=False)

Index(['20', '20', '20', '20', '20', '20', '20', '20', '20', '20', '20', '20',
       '20', '20', '20', '20', '20', '20', '20', '20', '20', '20', '20', '20',
       '20', '20'],
      dtype='object')

A convenient feature that is built into regular expressions is the ability to name each group, so that you can access the matched pattern by name (via the .group attribute of a match object).

In [16]:
pat = '-(?P<month>\d\d)-'
m = re.search(pat, dser[0])
print(dser[0], m.group('month'), m.group(1))

2018-01-07 01 01


In the next example, we use the str.extract method to extract multiple grouped patterns and expand the result into a DataFrame, where each matched group is placed in a separate column. Named groups are especially convenient when using this approach, as the names of each group are assigned as column names in the returned DataFrame.

In [17]:
# str.extract - Multiple groups
dser.str.extract('(?P<year>\d\d\d\d)-(?P<month>\d\d)-(?P<day>\d\d)', expand=True).head()
# # expand = True --> dataframe, which is quite useful when we have 2 more groups 
# # expand = False --> array

,year,month,day
0,2018,01,07
1,2018,01,21
2,2018,02,04
3,2018,02,18
4,2018,03,04


In [18]:
# str.extractall
dser.str.extractall('(\d+)').unstack().loc[:5,0]

match,0,1,2
0,2018,01,07
1,2018,01,21
2,2018,02,04
3,2018,02,18
4,2018,03,04
5,2018,03,18


In [19]:
# str.findall
dser.str.findall('(\d+)')

Index([['2018', '01', '07'], ['2018', '01', '21'], ['2018', '02', '04'],
       ['2018', '02', '18'], ['2018', '03', '04'], ['2018', '03', '18'],
       ['2018', '04', '01'], ['2018', '04', '15'], ['2018', '04', '29'],
       ['2018', '05', '13'], ['2018', '05', '27'], ['2018', '06', '10'],
       ['2018', '06', '24'], ['2018', '07', '08'], ['2018', '07', '22'],
       ['2018', '08', '05'], ['2018', '08', '19'], ['2018', '09', '02'],
       ['2018', '09', '16'], ['2018', '09', '30'], ['2018', '10', '14'],
       ['2018', '10', '28'], ['2018', '11', '11'], ['2018', '11', '25'],
       ['2018', '12', '09'], ['2018', '12', '23']],
      dtype='object')

In [20]:
# str.match
dser.str.match('-\d\d')

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False])

In [21]:
# str.replace
dser.str.replace('-0','-')

Index(['2018-1-7', '2018-1-21', '2018-2-4', '2018-2-18', '2018-3-4',
       '2018-3-18', '2018-4-1', '2018-4-15', '2018-4-29', '2018-5-13',
       '2018-5-27', '2018-6-10', '2018-6-24', '2018-7-8', '2018-7-22',
       '2018-8-5', '2018-8-19', '2018-9-2', '2018-9-16', '2018-9-30',
       '2018-10-14', '2018-10-28', '2018-11-11', '2018-11-25', '2018-12-9',
       '2018-12-23'],
      dtype='object')

In [22]:
# str.split
dser.str.split('-|/')

Index([['2018', '01', '07'], ['2018', '01', '21'], ['2018', '02', '04'],
       ['2018', '02', '18'], ['2018', '03', '04'], ['2018', '03', '18'],
       ['2018', '04', '01'], ['2018', '04', '15'], ['2018', '04', '29'],
       ['2018', '05', '13'], ['2018', '05', '27'], ['2018', '06', '10'],
       ['2018', '06', '24'], ['2018', '07', '08'], ['2018', '07', '22'],
       ['2018', '08', '05'], ['2018', '08', '19'], ['2018', '09', '02'],
       ['2018', '09', '16'], ['2018', '09', '30'], ['2018', '10', '14'],
       ['2018', '10', '28'], ['2018', '11', '11'], ['2018', '11', '25'],
       ['2018', '12', '09'], ['2018', '12', '23']],
      dtype='object')

The str.get_dummies method is very useful for creating dummy variables from text data, but in general, the functionality is similar to the pd.get_dummies function.

In [23]:
# Import video games data
df = pd.read_csv('vgsales.csv', index_col=0)
df.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [24]:
df['Genre'].str.get_dummies().head()

,Action,Adventure,Fighting,Misc,Platform,Puzzle,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy
Rank,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,1,0
5,0,0,0,0,0,0,0,1,0,0,0,0


The real value in the vectorized string method of get_dummies is if you have multiple categories concatenated together with a delimiter. For example, consider the MovieLens 1M data from the examples from the text.

In [25]:
df = pd.read_csv('movielens/movies.dat', usecols=[1,2], sep='::', header=None, engine='python')
df.columns = ['Title', 'Genres']
df.head()

,Title,Genres
0,Toy Story (1995),Animation|Children's|Comedy
1,Jumanji (1995),Adventure|Children's|Fantasy
2,Grumpier Old Men (1995),Comedy|Romance
3,Waiting to Exhale (1995),Comedy|Drama
4,Father of the Bride Part II (1995),Comedy


In [26]:
df['Genres'].str.get_dummies(sep='|').head()

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
df['Genres'].str.get_dummies(sep='|').sum().sort_values(ascending=False)

Drama          1603
Comedy         1200
Action          503
Thriller        492
Romance         471
Horror          343
Adventure       283
Sci-Fi          276
Children's      251
Crime           211
War             143
Documentary     127
Musical         114
Mystery         106
Animation       105
Western          68
Fantasy          68
Film-Noir        44
dtype: int64

What other processing tasks might we need to perform on this data???

### 2-Minute Activity

In [28]:
df.insert(1,'Year',df['Title'].str.extract('\((\d{4})\)', expand = False). astype(int).head())

In [29]:
df.head()

,Title,Year,Genres
0,Toy Story (1995),1995.0,Animation|Children's|Comedy
1,Jumanji (1995),1995.0,Adventure|Children's|Fantasy
2,Grumpier Old Men (1995),1995.0,Comedy|Romance
3,Waiting to Exhale (1995),1995.0,Comedy|Drama
4,Father of the Bride Part II (1995),1995.0,Comedy


In [30]:
df['Number of Genres'] = df['Genres'].str.split('|').str.len().head()
df.head()

,Title,Year,Genres,Number of Genres
0,Toy Story (1995),1995.0,Animation|Children's|Comedy,3.0
1,Jumanji (1995),1995.0,Adventure|Children's|Fantasy,3.0
2,Grumpier Old Men (1995),1995.0,Comedy|Romance,2.0
3,Waiting to Exhale (1995),1995.0,Comedy|Drama,2.0
4,Father of the Bride Part II (1995),1995.0,Comedy,1.0


In [31]:
df['Title'].str.contains('II$|III$|2$|3$').sum()

# # search at the end of string using $

0

## Additional Text Processing Examples

### Airline Mergers and Acquisitions

In [32]:
df = pd.read_html('http://airlines.org/dataset/u-s-airline-mergers-and-acquisitions/', header=0)[0]
df.head()

,Title,Announced,Closed,Resulting Entity
0,Western Air Express / Standard Airlines,NaN,5/1/1930,Western Air Express
1,Western Air Express / Transcontinental Air Transport,NaN,7/16/1930,Transcontinental & Western Air
2,Delta Air Lines / Chicago and Southern (C&S) Air Lines,NaN,5/1/1953,Delta Air Lines
3,United Airlines / Capital Airlines,NaN,6/1/1961,United Airlines
4,Western Airlines / Pacific Northern Airlines,NaN,7/1/1967,Western Airlines


In [33]:
# Most common airlines involved in M&A
df['Title'].str.get_dummies(sep=' / ').sum().sort_values(ascending=False).head(10)

Delta Air Lines         7
American Airlines       6
Alaska Airlines         5
Southwest Airlines      4
Continental Airlines    3
United Airlines         3
Republic Airways        3
Frontier Airlines       3
Western Air Express     2
Pinnacle Airlines       2
dtype: int64

In [34]:
# Extract years of M&A - Method 1
df['Closed'].str[-4:].head()

0    1930
1    1930
2    1953
3    1961
4    1967
Name: Closed, dtype: object

In [35]:
# Extract years of M&A - Method 2
df['Closed'].str.slice(-4).head()

0    1930
1    1930
2    1953
3    1961
4    1967
Name: Closed, dtype: object

In [36]:
# Extract years of M&A - Method 3
df['Closed'].str.extract('(\d{4})', expand=False).head()

0    1930
1    1930
2    1953
3    1961
4    1967
Name: Closed, dtype: object

In [37]:
# Detect variant of airlines in merged entity name
df['Resulting Entity'].str.contains('air ?lines', case=False).value_counts()

True     36
False    17
Name: Resulting Entity, dtype: int64

### WMATA Holiday Schedule

In [38]:
df = pd.read_html('https://www.wmata.com/schedules/timetables/index.cfm', match='Date', header=0)[0]
df

,Date,Holiday,Metrorail Schedule,Metrobus Schedule,Hours
0,"Tuesday, 12/25/18",Christmas Day,Sunday,Sunday,8 am - 11 pm
1,NaN,NaN,NaN,NaN,NaN
2,"Tuesday, 1/1/19",New Years Day,Sunday,Sunday,8 am - 11 pm
3,"Monday, 1/21/19",Martin Luther King Jr. Day,Saturday,Saturday + Supplemental,5 am - 11:30 pm
4,"Monday, 2/18/19",Presidents Day,Saturday,Saturday + Supplemental,5 am - 11:30 pm
5,"Monday, 5/27/19",Memorial Day,Sunday,Sunday,8 am - 11 pm
6,"Thursday, 7/04/19",Independence Day,Special,Saturday,7 am - 11:30 pm
7,"Monday, 9/2/19",Labor Day,Sunday,Sunday,8 am - 11 pm
8,"Monday, 10/14/19",Columbus Day,Saturday,Saturday + Supplemental,5 am - 11:30 pm
9,"Monday, 11/11/19",Veterans Day,Saturday,Saturday + Supplemental,5 am - 11:30 pm


In [39]:
# Insert day of the week
df.insert(0, 'DOW', df['Date'].str.split(', ').str[0])
df

,DOW,Date,Holiday,Metrorail Schedule,Metrobus Schedule,Hours
0,Tuesday,"Tuesday, 12/25/18",Christmas Day,Sunday,Sunday,8 am - 11 pm
1,NaN,NaN,NaN,NaN,NaN,NaN
2,Tuesday,"Tuesday, 1/1/19",New Years Day,Sunday,Sunday,8 am - 11 pm
3,Monday,"Monday, 1/21/19",Martin Luther King Jr. Day,Saturday,Saturday + Supplemental,5 am - 11:30 pm
4,Monday,"Monday, 2/18/19",Presidents Day,Saturday,Saturday + Supplemental,5 am - 11:30 pm
5,Monday,"Monday, 5/27/19",Memorial Day,Sunday,Sunday,8 am - 11 pm
6,Thursday,"Thursday, 7/04/19",Independence Day,Special,Saturday,7 am - 11:30 pm
7,Monday,"Monday, 9/2/19",Labor Day,Sunday,Sunday,8 am - 11 pm
8,Monday,"Monday, 10/14/19",Columbus Day,Saturday,Saturday + Supplemental,5 am - 11:30 pm
9,Monday,"Monday, 11/11/19",Veterans Day,Saturday,Saturday + Supplemental,5 am - 11:30 pm


In [40]:
# Remove DOW from date column
df['Date'] = df['Date'].str.split(', ').str[1]
df

,DOW,Date,Holiday,Metrorail Schedule,Metrobus Schedule,Hours
0,Tuesday,12/25/18,Christmas Day,Sunday,Sunday,8 am - 11 pm
1,NaN,NaN,NaN,NaN,NaN,NaN
2,Tuesday,1/1/19,New Years Day,Sunday,Sunday,8 am - 11 pm
3,Monday,1/21/19,Martin Luther King Jr. Day,Saturday,Saturday + Supplemental,5 am - 11:30 pm
4,Monday,2/18/19,Presidents Day,Saturday,Saturday + Supplemental,5 am - 11:30 pm
5,Monday,5/27/19,Memorial Day,Sunday,Sunday,8 am - 11 pm
6,Thursday,7/04/19,Independence Day,Special,Saturday,7 am - 11:30 pm
7,Monday,9/2/19,Labor Day,Sunday,Sunday,8 am - 11 pm
8,Monday,10/14/19,Columbus Day,Saturday,Saturday + Supplemental,5 am - 11:30 pm
9,Monday,11/11/19,Veterans Day,Saturday,Saturday + Supplemental,5 am - 11:30 pm


In [41]:
# Add open and close column, delete Hours column
df['Open'] = df['Hours'].str.split(' - ').str[0]
df['Close'] = df['Hours'].str.split(' - ').str[1]
del df['Hours']

In [42]:
# Output clean data to file and display final result
df.to_csv('wmata_holidays.csv', index=False)
df

,DOW,Date,Holiday,Metrorail Schedule,Metrobus Schedule,Open,Close
0,Tuesday,12/25/18,Christmas Day,Sunday,Sunday,8 am,11 pm
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Tuesday,1/1/19,New Years Day,Sunday,Sunday,8 am,11 pm
3,Monday,1/21/19,Martin Luther King Jr. Day,Saturday,Saturday + Supplemental,5 am,11:30 pm
4,Monday,2/18/19,Presidents Day,Saturday,Saturday + Supplemental,5 am,11:30 pm
5,Monday,5/27/19,Memorial Day,Sunday,Sunday,8 am,11 pm
6,Thursday,7/04/19,Independence Day,Special,Saturday,7 am,11:30 pm
7,Monday,9/2/19,Labor Day,Sunday,Sunday,8 am,11 pm
8,Monday,10/14/19,Columbus Day,Saturday,Saturday + Supplemental,5 am,11:30 pm
9,Monday,11/11/19,Veterans Day,Saturday,Saturday + Supplemental,5 am,11:30 pm


### Top 30 Airports

In [43]:
df = pd.read_html('https://www.world-airport-codes.com/world-top-30-airports.html', header=0, index_col=0)[0]
df.head()

,Airport,Location,Country,Code(IATA/ICAO),Totalpassengers,RankChange,%Change
Rank,,,,,,,
1.0,Hartsfield–Jackson Atlanta International Airport,"Atlanta, Georgia",United States,ATL/KATL,107394029,NaN,3.3%
2.0,Beijing Capital International Airport,"Chaoyang-Shunyi, Beijing",China,PEK/ZBAA,100983290,NaN,5.4%
3.0,Dubai International Airport,"Garhoud, Dubai",United Arab Emirates,DXB/OMDB,89149387,NaN,1.0%
4.0,Los Angeles International Airport,"Los Angeles, California",United States,LAX/KLAX,87534384,1.0,3.5%
5.0,Tokyo Haneda Airport,"Ōta, Tokyo",Japan,HND/RJTT,87131973,1.0,2.0%


In [44]:
# Extract city/metropolitan area
df.insert(1, 'Metro', df['Location'].str.split(', ').str[0])
df.head()

,Airport,Metro,Location,Country,Code(IATA/ICAO),Totalpassengers,RankChange,%Change
Rank,,,,,,,,
1.0,Hartsfield–Jackson Atlanta International Airport,Atlanta,"Atlanta, Georgia",United States,ATL/KATL,107394029,NaN,3.3%
2.0,Beijing Capital International Airport,Chaoyang-Shunyi,"Chaoyang-Shunyi, Beijing",China,PEK/ZBAA,100983290,NaN,5.4%
3.0,Dubai International Airport,Garhoud,"Garhoud, Dubai",United Arab Emirates,DXB/OMDB,89149387,NaN,1.0%
4.0,Los Angeles International Airport,Los Angeles,"Los Angeles, California",United States,LAX/KLAX,87534384,1.0,3.5%
5.0,Tokyo Haneda Airport,Ōta,"Ōta, Tokyo",Japan,HND/RJTT,87131973,1.0,2.0%


In [45]:
# Extract state/province
df.insert(2, 'State', df['Location'].str.extract(', (.+)', expand=False))
del df['Location']
df.head()

,Airport,Metro,State,Country,Code(IATA/ICAO),Totalpassengers,RankChange,%Change
Rank,,,,,,,,
1.0,Hartsfield–Jackson Atlanta International Airport,Atlanta,Georgia,United States,ATL/KATL,107394029,NaN,3.3%
2.0,Beijing Capital International Airport,Chaoyang-Shunyi,Beijing,China,PEK/ZBAA,100983290,NaN,5.4%
3.0,Dubai International Airport,Garhoud,Dubai,United Arab Emirates,DXB/OMDB,89149387,NaN,1.0%
4.0,Los Angeles International Airport,Los Angeles,California,United States,LAX/KLAX,87534384,1.0,3.5%
5.0,Tokyo Haneda Airport,Ōta,Tokyo,Japan,HND/RJTT,87131973,1.0,2.0%


In [46]:
# Split and expand IATA and ICAO airport codes
codes = df['Code(IATA/ICAO)'].str.split('/', expand=True).head()
codes.columns = ['IATA', 'ICAO']
codes.head()

,IATA,ICAO
Rank,,
1.0,ATL,KATL
2.0,PEK,ZBAA
3.0,DXB,OMDB
4.0,LAX,KLAX
5.0,HND,RJTT


In [47]:
# Insert codes into DataFrame
df.insert(4, 'IATA', codes['IATA'])
df.insert(5, 'ICAO', codes['ICAO'])
del df['Code(IATA/ICAO)']
df.head()

,Airport,Metro,State,Country,IATA,ICAO,Totalpassengers,RankChange,%Change
Rank,,,,,,,,,
1.0,Hartsfield–Jackson Atlanta International Airport,Atlanta,Georgia,United States,ATL,KATL,107394029,NaN,3.3%
2.0,Beijing Capital International Airport,Chaoyang-Shunyi,Beijing,China,PEK,ZBAA,100983290,NaN,5.4%
3.0,Dubai International Airport,Garhoud,Dubai,United Arab Emirates,DXB,OMDB,89149387,NaN,1.0%
4.0,Los Angeles International Airport,Los Angeles,California,United States,LAX,KLAX,87534384,1.0,3.5%
5.0,Tokyo Haneda Airport,Ōta,Tokyo,Japan,HND,RJTT,87131973,1.0,2.0%


In [48]:
# Scrape country codes
countries = pd.read_html('https://www.worldatlas.com/aatlas/ctycodes.htm', header=0)[0]
countries.rename(columns=lambda s: s.split(' ')[0].capitalize(), inplace=True)
countries.head()

,Country,A2,A3,Num,Dialing
0,Afghanistan,AF,AFG,4,93
1,Albania,AL,ALB,8,355
2,Algeria,DZ,DZA,12,213
3,American Samoa,AS,ASM,16,1-684
4,Andorra,AD,AND,20,376


In [49]:
# Define country map and apply to Country column of DataFrame
country_map = dict(zip(countries['Country'], countries['A3']))
df['Country'] = df['Country'].map(country_map)
df.head()

,Airport,Metro,State,Country,IATA,ICAO,Totalpassengers,RankChange,%Change
Rank,,,,,,,,,
1.0,Hartsfield–Jackson Atlanta International Airport,Atlanta,Georgia,USA,ATL,KATL,107394029,NaN,3.3%
2.0,Beijing Capital International Airport,Chaoyang-Shunyi,Beijing,CHN,PEK,ZBAA,100983290,NaN,5.4%
3.0,Dubai International Airport,Garhoud,Dubai,ARE,DXB,OMDB,89149387,NaN,1.0%
4.0,Los Angeles International Airport,Los Angeles,California,USA,LAX,KLAX,87534384,1.0,3.5%
5.0,Tokyo Haneda Airport,Ōta,Tokyo,JPN,HND,RJTT,87131973,1.0,2.0%


In [50]:
# Check dtypes
df.dtypes

Airport             object
Metro               object
State               object
Country             object
IATA                object
ICAO                object
Totalpassengers      int64
RankChange         float64
%Change             object
dtype: object

In [51]:
# Output data to file
df.to_csv('top_airports.csv', index=True)

## Next Time: Merging and Reshaping Data